In [4]:
import pandas as pd
from birdclef.utils import slice_seconds
import librosa

df = pd.read_parquet("../data/intermediate/2022-04-03-motif-consolidated.parquet")

In [17]:
df.columns

Index(['source_name', 'cens_sample_rate', 'matrix_profile_window', 'motif_0',
       'motif_1', 'sample_rate', 'duration_cens', 'duration_samples',
       'duration_seconds', 'mp', 'pi'],
      dtype='object')

In [18]:
row = df[df.duration_seconds > 20].iloc[0]
row

source_name                               train_audio/afrsil1/XC175522.ogg
cens_sample_rate                                                        10
matrix_profile_window                                                   50
motif_0                                                              229.0
motif_1                                                              305.0
sample_rate                                                          22050
duration_cens                                                          477
duration_samples                                                   1036800
duration_seconds                                                     47.02
mp                       [0.6818745909002342, 0.6721157504130568, 0.667...
pi                       [119, 119, 119, 119, 119, 119, 321, 322, 322, ...
Name: 1, dtype: object

In [19]:
sr = 32000
y, _ = librosa.load(f"../data/raw/birdclef-2022/{row.source_name}", sr=sr)
y.shape

(1504653,)

In [20]:
sliced = slice_seconds(y, sr, 5)
sliced

[(0,
  array([ 6.8260611e-06,  4.5988859e-06, -6.1190517e-06, ...,
         -7.5683449e-03, -9.9657802e-03, -1.1053292e-02], dtype=float32)),
 (5,
  array([-0.00929923, -0.00829683, -0.0087063 , ..., -0.00422795,
          0.00385742,  0.01366972], dtype=float32)),
 (10,
  array([0.01968161, 0.02466812, 0.02736103, ..., 0.00382465, 0.00378051,
         0.00376146], dtype=float32)),
 (15,
  array([0.0046633 , 0.00267117, 0.00056672, ..., 0.01335949, 0.01530494,
         0.00337507], dtype=float32)),
 (20,
  array([-0.01608314, -0.0251033 , -0.00948783, ..., -0.00217814,
          0.001242  ,  0.00411835], dtype=float32)),
 (25,
  array([ 0.00528328,  0.00674923,  0.00549289, ..., -0.00106612,
         -0.00192043, -0.00208457], dtype=float32)),
 (30,
  array([0.00158561, 0.00216993, 0.00260077, ..., 0.00689777, 0.00931174,
         0.00316191], dtype=float32)),
 (35,
  array([-1.92809524e-03, -3.45244072e-03, -9.32841445e-04, ...,
          5.87553834e-04, -1.34750735e-05, -2.13201638e-

In [22]:
row.pi[:20]

array([119, 119, 119, 119, 119, 119, 321, 322, 322,  84,  85,  86,  87,
        88,  89,  90,  90,  91,  92,  93], dtype=int64)

So, it turns out that the profile index maps into the cens windows and not the second windows. We have to divide this by 50 in order to get the appropriate index...

In [32]:
from scipy.stats import mode
from birdclef.utils import chunks

for chunk in chunks(row.pi // 50, 50):
    print(mode(chunk)[0][0])

2
0
6
0
6
6
4
2
2
